# Cohort Analysis
This notebook includes EDA, cohort analysis calculation.    
Snowpark session Documentation : https://docs.snowflake.com/ko/developer-guide/snowpark/reference/python/session.html    
Python Documentation: https://docs.snowflake.com/en/developer-guide/snowpark/python/index    

**examples**
Github example:https://github.com/Snowflake-Labs/snowpark-python-demos/tree/main/Advertising-Spend-ROI-Prediction    
Jupyter notebook example: https://github.com/Snowflake-Labs/snowpark-python-demos/blob/main/Advertising-Spend-ROI-Prediction/Snowpark_For_Python.ipynb

#### Connect to Snowflake

In [1]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit,count
from snowflake.snowpark.version import VERSION
# Misc
import json
import pandas as pd
import numpy as np
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

In [2]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(snowflake_environment[0][1]))
print('Database                    : {}'.format(snowflake_environment[0][2]))
print('Schema                      : {}'.format(snowflake_environment[0][3]))
print('Warehouse                   : {}'.format(snowflake_environment[0][5]))
print('Snowflake version           : {}'.format(snowflake_environment[0][4]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : ESTPEGION
Role                        : ACCOUNTADMIN
Database                    : KPMG
Schema                      : PUBLIC
Warehouse                   : COMPUTE_WH
Snowflake version           : 7.6.2
Snowpark for Python version : 1.1.0


#### List all tables in the schema

In [22]:
session.sql('select TABLE_NAME from information_schema.tables').show()

-------------------
|"TABLE_NAME"     |
-------------------
|CUSTOMERADDRESS  |
|FOOD             |
|TRANSACTIONS     |
|DEMOGRAPHIC      |
|NEWCUSTOMER      |
|TABLES           |
|COLUMNS          |
|SCHEMATA         |
|SEQUENCES        |
|VIEWS            |
-------------------



#### Query table from snowflake

In [12]:
snow_df_spend = session.table('TRANSACTIONS')
snow_df_spend.show()

2023-02-22 14:18:03.064 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM TRANSACTIONS LIMIT 10]
2023-02-22 14:18:03.159 INFO    snowflake.connector.cursor: query execution done


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRANSACTION_ID"  |"PRODUCT_ID"  |"CUSTOMER_ID"  |"TRANSACTION_DATE"   |"ONLINE_ORDER"  |"ORDER_STATUS"  |"BRAND"         |"PRODUCT_LINE"  |"PRODUCT_CLASS"  |"PRODUCT_SIZE"  |"LIST_PRICE"  |"STANDARD_COST"  |"PRODUCT_FIRST_SOLD_DATE"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1                 |2             |2950           |2017-02-25 00:00:00  |0.0             |Approved        |Solex           |Standard        |medium           |medium          |71.49         |53.62            |41245.0                    |
|2                 |3             |3120         

In [24]:
# validate row number
snow_df_spend.select(count('*')).show()

--------------
|"COUNT(1)"  |
--------------
|20000       |
--------------



In [13]:
%%writefile main.py
import streamlit as st
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import datetime as dt
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit,count
from snowflake.snowpark.version import VERSION
# Misc
import json
import logging 

# The code below is for the title and logo for this page.
st.set_page_config(page_title="Cohort Analysis on the Bikes dataset", page_icon="🚲")
bg_image = '''
<style>
[data-testid="stAppViewContainer"]{
background-image: url(https://lp-cms-production.imgix.net/2022-01/GettyRF_475680439.jpg?auto=format&q=75&w=1920);
backgroud-size:cover;
}

[data-testid="stHeader"]{
background-color: rgba(0,0,0,0);
}

[data-testid="stToolbar"]{
right : 2rem;}

</style>
'''
#st.markdown(bg_image, unsafe_allow_html=True)
#st.set_page_config(layout="wide")
st.sidebar.markdown("# Bike Cohort Analysis")
st.image(
    "TrevelBike.jpeg",
    #width  = 160
    use_column_width = True,
)

st.title("Cohort Analysis → `Bikes` dataset")

st.write("")

st.markdown(
    """

    This demo is inspired by this [Cohort Analysis Tutorial](https://github.com/maladeep/cohort-retention-rate-analysis-in-python).

"""
)

with st.expander("About this app"):

    st.write("")

    st.markdown(
        """

    This dataset comes from the hypothetical KPMG.

    Each row in the dataset contains information about an individual bike purchase:

    - Who bought it
    - How much they paid
    - The bike's `brand` and `product line`
    - Its `class` and `size`
    - What day the purchase happened
    - The day the product was first sold
    """
    )

    st.write("")

    st.markdown(
        """
    The underlying code groups those purchases into cohorts and calculates the `retention rate` (split by month) so that one can answer the question:

    *if I'm making monthly changes to my store to get people to come back and buy more bikes, are those changes working?"*

    These cohorts are then visualized and interpreted through a heatmap [powered by Plotly](https://plotly.com/python/).

    """
    )

    st.write("")

# A function that will parse the date Time based cohort:  1 day of month
def get_month(x):
    return dt.datetime(x.year, x.month, 1)

@st.cache_resource
def connect2snowflake():
    # set logger
    logger = logging.getLogger("snowflake.snowpark.session")
    logger.setLevel(logging.ERROR)
    # Create Snowflake Session object
    connection_parameters = json.load(open('connection.json'))
    session = Session.builder.configs(connection_parameters).create()
    session.sql_simplifier_enabled = True

    snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
    snowpark_version = VERSION
    return session
session = connect2snowflake()

@st.cache_data
def load_data():

    # Load data
    transaction_df = pd.DataFrame(session.table('TRANSACTIONS').collect())
    #transaction_df = session.sql('select * from TRANSACTIONS').toPandas()
    transaction_df.columns = [x.lower() for x in transaction_df.columns]


    # Process data
    transaction_df = transaction_df.replace(" ", np.NaN)
    transaction_df = transaction_df.fillna(transaction_df.mean())
    transaction_df["TransactionMonth"] = transaction_df["transaction_date"].apply(
        get_month
    )
    transaction_df["TransactionYear"] = transaction_df["transaction_date"].dt.year
    transaction_df["TransactionMonth"] = transaction_df["transaction_date"].dt.month
    for col in transaction_df.columns:
        if transaction_df[col].dtype == "object":
            transaction_df[col] = transaction_df[col].fillna(
                transaction_df[col].value_counts().index[0]
            )

    # Create transaction_date column based on month and store in TransactionMonth
    transaction_df["TransactionMonth"] = transaction_df["transaction_date"].apply(
        get_month
    )
    # Grouping by customer_id and select the InvoiceMonth value
    grouping = transaction_df.groupby("customer_id")["TransactionMonth"]
    # Assigning a minimum InvoiceMonth value to the dataset
    transaction_df["CohortMonth"] = grouping.transform("min")

    return transaction_df


transaction_df = load_data()

with st.expander("Show the `Bikes` dataframe"):
    st.write(transaction_df)


def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day


# Getting the integers for date parts from the `InvoiceDay` column
transcation_year, transaction_month, _ = get_date_int(
    transaction_df, "TransactionMonth"
)
# Getting the integers for date parts from the `CohortDay` column
cohort_year, cohort_month, _ = get_date_int(transaction_df, "CohortMonth")
#  Get the  difference in years
years_diff = transcation_year - cohort_year
# Calculate difference in months
months_diff = transaction_month - cohort_month

# Extract the difference in months from all previous values "+1" in addeded at the end so that first month is marked as 1 instead of 0 for easier interpretation. """
transaction_df["CohortIndex"] = years_diff * 12 + months_diff + 1

dtypes = transaction_df.dtypes.astype(str)
# Show dtypes
# dtypes

transaction_df_new_slider_01 = transaction_df[["brand", "product_line"]]
new_slider_01 = [col for col in transaction_df_new_slider_01]

transaction_df_new_slider_02 = transaction_df[["list_price", "standard_cost"]]
new_slider_02 = [col for col in transaction_df_new_slider_02]

st.write("")

cole, col1, cole, col2, cole = st.columns([0.1, 1, 0.05, 1, 0.1])

with col1:

    MetricSlider01 = st.selectbox("Pick your 1st metric", new_slider_01)

    MetricSlider02 = st.selectbox("Pick your 2nd metric", new_slider_02, index=1)

    st.write("")

with col2:

    if MetricSlider01 == "brand":
        # col_one_list = transaction_df_new["brand"].tolist()
        col_one_list = transaction_df_new_slider_01["brand"].drop_duplicates().tolist()
        multiselect = st.multiselect(
            "Select the value(s)", col_one_list, ["Solex", "Trek Bicycles"]
        )
        transaction_df = transaction_df[transaction_df["brand"].isin(multiselect)]

    elif MetricSlider01 == "product_line":
        col_one_list = (
            transaction_df_new_slider_01["product_line"].drop_duplicates().tolist()
        )
        multiselect = st.multiselect(
            "Select the value(s)", col_one_list, ["Standard", "Road"]
        )
        transaction_df = transaction_df[
            transaction_df["product_line"].isin(multiselect)
        ]

    if MetricSlider02 == "list_price":
        list_price_slider = st.slider(
            "List price (in $)", step=500, min_value=12, max_value=2091
        )
        transaction_df = transaction_df[
            transaction_df["list_price"] > list_price_slider
        ]

    elif MetricSlider02 == "standard_cost":
        standard_cost_slider = st.slider(
            "Standard cost (in $)", step=500, min_value=7, max_value=1759
        )
        transaction_df = transaction_df[
            transaction_df["list_price"] > standard_cost_slider
        ]

try:

    # Counting daily active user from each chort
    grouping = transaction_df.groupby(["CohortMonth", "CohortIndex"])
    # Counting number of unique customer Id's falling in each group of CohortMonth and CohortIndex
    cohort_data = grouping["customer_id"].apply(pd.Series.nunique)
    cohort_data = cohort_data.reset_index()
    # Assigning column names to the dataframe created above
    cohort_counts = cohort_data.pivot(
        index="CohortMonth", columns="CohortIndex", values="customer_id"
    )

    cohort_sizes = cohort_counts.iloc[:, 0]
    retention = cohort_counts.divide(cohort_sizes, axis=0)
    # Coverting the retention rate into percentage and Rounding off.
    retention = retention.round(3) * 100
    retention.index = retention.index.strftime("%Y-%m")

    # Plotting the retention rate
    fig = go.Figure()

    fig.add_heatmap(
        # x=retention.columns, y=retention.index, z=retention, colorscale="cividis"
        x=retention.columns,
        y=retention.index,
        z=retention,
        # Best
        # colorscale="Aggrnyl",
        colorscale="Bluyl",
    )

    fig.update_layout(title_text="Monthly cohorts showing retention rates", title_x=0.2)
    fig.layout.xaxis.title = "Cohort Group"
    fig.layout.yaxis.title = "Cohort Period"
    fig["layout"]["title"]["font"] = dict(size=25)
    fig.layout.width = 750
    fig.layout.height = 750
    fig.layout.xaxis.tickvals = retention.columns
    fig.layout.yaxis.tickvals = retention.index
    fig.layout.plot_bgcolor = "#efefef"  # Set the background color to white
    fig.layout.margin.b = 100
    fig

except IndexError:
    st.warning("This is throwing an exception, bear with us!")

Overwriting main.py


In [33]:
%%writefile pages/Food_Cohort_Analysis.py
import streamlit as st
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import datetime as dt
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit,count
from snowflake.snowpark.version import VERSION
# Misc
import json
import logging 

# The code below is for the title and logo for this page.
st.set_page_config(page_title="Cohort Analysis on the Food dataset", page_icon="🍔")

st.image(
    "Food.jpg",
    use_column_width = True,
)

st.title("Cohort Analysis → `Food` dataset")

st.write("")



with st.expander("About this app"):

    st.write("")

    st.markdown(
        """

    This dataset comes from the hypothetical KPMG.

    Each row in the dataset contains information about an individual bike purchase:

    - Who bought it
    - How much they paid
    - The bike's `brand` and `product line`
    - Its `class` and `size`
    - What day the purchase happened
    - The day the product was first sold
    """
    )

    st.write("")

    st.markdown(
        """
    The underlying code groups those purchases into cohorts and calculates the `retention rate` (split by month) so that one can answer the question:

    *if I'm making monthly changes to my store to get people to come back and buy more bikes, are those changes working?"*

    These cohorts are then visualized and interpreted through a heatmap [powered by Plotly](https://plotly.com/python/).

    """
    )

    st.write("")

# A function that will parse the date Time based cohort:  1 day of month
def get_month(x):
    return dt.datetime(x.year, x.month, 1)

@st.cache_resource
def connect2snowflake():
    # set logger
    logger = logging.getLogger("snowflake.snowpark.session")
    logger.setLevel(logging.ERROR)
    # Create Snowflake Session object
    connection_parameters = json.load(open('connection.json'))
    session = Session.builder.configs(connection_parameters).create()
    session.sql_simplifier_enabled = True

    snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
    snowpark_version = VERSION
    return session
session = connect2snowflake()



@st.cache_data
def load_data():

    # Load data
    food_df = pd.DataFrame(session.table('FOOD').collect())
    food_df['ORDERDATE'] = pd.to_datetime(food_df['ORDERDATE'])
    #food_df['PICKUPDATE'] = pd.to_datetime(food_df['PICKUPDATE'])
    food_df = food_df.replace(" ",np.NaN)
    food_df = food_df.fillna(food_df.mean(numeric_only=True))
    #food_df.isna().sum()
    food_df['TransactionMonth'] =food_df['ORDERDATE'].apply(lambda x: x.replace(day = 1))
    # Grouping by customer_id and select the InvoiceMonth value
    grouping = food_df.groupby('USERID')['TransactionMonth'] 

    # Assigning a minimum InvoiceMonth value to the dataset
    food_df['CohortMonth'] = grouping.transform('min')
    



    return food_df


food_df = load_data()

with st.expander("Show the `Food` dataframe"):
    st.write(food_df)

@st.cache_data
def get_minmaxCharges():
    return food_df.TOTALCHARGES.max(), food_df.TOTALCHARGES.min()

max_charge, min_charge = get_minmaxCharges()
_, col1, _ = st.columns([0.2, 1, 0.2])

with col1:
    TotalCharges_slider = st.slider(
            "Total Charges (in $)",  0.0,  max_charge, 
        )    

food_df = food_df[food_df["TOTALCHARGES"] > TotalCharges_slider]
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day
# Getting the integers for date parts from the `InvoiceDay` column
transcation_year, transaction_month, _ = get_date_int(food_df, 'TransactionMonth')

# Getting the integers for date parts from the `CohortDay` column
cohort_year, cohort_month, _ = get_date_int(food_df, 'CohortMonth')
#  Get the  difference in years
years_diff = transcation_year - cohort_year

# Calculate difference in months
months_diff = transaction_month - cohort_month
food_df['CohortIndex'] = years_diff * 12 + months_diff  + 1 
# Counting daily active user from each chort
grouping = food_df.groupby(['CohortMonth', 'CohortIndex'])



# Counting number of unique customer Id's falling in each group of CohortMonth and CohortIndex
cohort_data = grouping['USERID'].apply(pd.Series.nunique)
cohort_data = cohort_data.reset_index()


 # Assigning column names to the dataframe created above
cohort_counts = cohort_data.pivot(index='CohortMonth',
                                 columns ='CohortIndex',
                                 values = 'USERID')
cohort_sizes = cohort_counts.iloc[:,0]
retention = cohort_counts.divide(cohort_sizes, axis=0)
retention.round(3)*100





Overwriting pages/Food_Cohort_Analysis.py


690.98

In [47]:
%%writefile environment.yml
name: SNPark
dependencies:
  - snowflake-snowpark-python
  - numpy
  - pandas
  - plotly
  - matplotlib
  - seaborn


Overwriting environment.yml


In [6]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import datetime as dt
def get_month(x):
    return dt.datetime(x.year, x.month, 1)
def load_data():

    # Load data
    transaction_df = pd.DataFrame(session.table('TRANSACTIONS').collect())
    #transaction_df = session.sql('select * from TRANSACTIONS').toPandas()
    transaction_df.columns = [x.lower() for x in transaction_df.columns]


    # Process data
    transaction_df = transaction_df.replace(" ", np.NaN)
    transaction_df = transaction_df.fillna(transaction_df.mean())
    transaction_df["TransactionMonth"] = transaction_df["transaction_date"].apply(
        get_month
    )
    transaction_df["TransactionYear"] = transaction_df["transaction_date"].dt.year
    transaction_df["TransactionMonth"] = transaction_df["transaction_date"].dt.month
    for col in transaction_df.columns:
        if transaction_df[col].dtype == "object":
            transaction_df[col] = transaction_df[col].fillna(
                transaction_df[col].value_counts().index[0]
            )

    # Create transaction_date column based on month and store in TransactionMonth
    transaction_df["TransactionMonth"] = transaction_df["transaction_date"].apply(
        get_month
    )
    # Grouping by customer_id and select the InvoiceMonth value
    grouping = transaction_df.groupby("customer_id")["TransactionMonth"]
    # Assigning a minimum InvoiceMonth value to the dataset
    transaction_df["CohortMonth"] = grouping.transform("min")

    return transaction_df

In [7]:
transaction_df = load_data()

/var/folders/p_/2pq06n0j76q0dwfrs31qvg240000gn/T/ipykernel_5903/3127388001.py:17: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  transaction_df = transaction_df.fillna(transaction_df.mean())
/var/folders/p_/2pq06n0j76q0dwfrs31qvg240000gn/T/ipykernel_5903/3127388001.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  transaction_df = transaction_df.fillna(transaction_df.mean())
